In [153]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [167]:
lib.utils.set_seed()
train_x, train_y = lib.datasets.get_linear2()
network = lib.nets.create_livenet_linear2(l1=0.1)
# network = lib.nets.LINEAR3(l1=0.1)

In [168]:
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=len(train_x))
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=30, adaptive_lr=False)


LiveNet


In [171]:

trainer.step(200)


Iˈ0.000 0.568 = 0.221+0.347 lib/trainer.py:89
Iˈ0.033 0.565 = 0.220+0.344 lib/trainer.py:89
Iˈ0.059 0->2 died lib/livenet.py:167
Iˈ0.059 DestinationNeuron 2 became dangle lib/death.py:27
Iˈ0.059 DataNeuron 0 became useless lib/livenet.py:58
Iˈ0.061 0.563 = 0.209+0.354 lib/trainer.py:89
Iˈ0.083 0.563 = 0.205+0.358 lib/trainer.py:89
Iˈ0.102 0.563 = 0.204+0.359 lib/trainer.py:89
Iˈ0.121 0.563 = 0.204+0.359 lib/trainer.py:89


In [152]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.0000

In [6]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_picker0(param):
    val0 = param["params"]["linear1.weight"][0][0].item()
    val1 = param["params"]["linear1.weight"][1][0].item()
    return val1 + val0

def param_picker1(param):
    val0 = param["params"]["1->2"].item()
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_picker1)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
# accum.plot()

Using matplotlib backend: TkAgg


In [172]:
for name, p in network.named_parameters():
    print(name, f"{p.item():.6f}")

2 0.927505
3 -0.946524
0->2 0.007142
0->3 -0.750392
1->2 -0.771363
1->3 3.772771
